# UDACITY : Designing Your First Workflow - Invoking Lambda Functions

In the last exercise, you created your own Lambda function. Without realizing it, you've already practiced invoking as well! Launching a test event is an example of synchronous invocation. In this exercise, you will continue working on the lambda function 'PreprocessLambda' from the previous exercise. However, you'll practice a different way to launch asynchronous invocation, and also practice the setup of an asynchronous invocation.
These are only two examples. Lambda is one of the most flexible offerings in AWS and can be utilized in a variety of applications. The same Lambda function can be (and often is) both invoked synchronously and asynchronously.

## Exercise: Synchronous invocation 

Synchronous invocations occur when a call is made to a Lambda function and a response is waited for. The example we're asking you to implement is a CLI invocation, but Lambda functions can also be placed behind Amazon's API Gateway for potential users to directly invoke a Lambda function. This, in turn, could be the interface that you expose to users to allow them to interact with other AWS resources. These types of invocations are great for "get" and "set" methods.

Your task is to synchronously invoke the Lambda function you implemented in the last exercise using the CLI. The following documentation may be useful to you: https://docs.aws.amazon.com/lambda/latest/dg/invocation-sync.html 

You will need to attach the LambdaFullAccess policy to the SageMaker execution role used for your notebook. Once done, it will take a few minutes for the policy to register.  

In [1]:
%%bash 
aws lambda invoke --function-name PreprocessLambda --payload '{"s3-dataset-uri": "udacity-ml-workflow/Lesson 3, Exercise 1/reviews_Patio_Lawn_and_Garden_5.json.zip"}' response.json

{
    "StatusCode": 200,
    "ExecutedVersion": "$LATEST"
}


Next task is to synchronously invoke the same Lambda function through SDK using boto3. The following documentation may be useful to you: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda/client/invoke.html

In [4]:
import boto3
from sagemaker import get_execution_role
import json

client = boto3.client('lambda')
payload = {"s3-dataset-uri": "udacity-ml-workflow/Lesson 3, Exercise 1/reviews_Patio_Lawn_and_Garden_5.json.zip"}

# json.dumps turns a JSON-object-like python object into a string, and .encode('utf-8') encodes the 
# the string so that it can be properly passed to the client. 

payload_bytes = json.dumps(payload).encode('utf-8')

response = client.invoke(FunctionName='PreprocessLambda', InvocationType='RequestResponse', Payload=payload_bytes)
response

{'ResponseMetadata': {'RequestId': '545d435f-43b3-4d39-a610-ce2df87e241a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 02 Sep 2023 23:17:09 GMT',
   'content-type': 'application/json',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '545d435f-43b3-4d39-a610-ce2df87e241a',
   'x-amzn-remapped-content-length': '0',
   'x-amz-executed-version': '$LATEST',
   'x-amzn-trace-id': 'root=1-64f3c26d-1d916f0c7a9609402d6429ff;sampled=0;lineage=923280ba:0'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'ExecutedVersion': '$LATEST',
 'Payload': <botocore.response.StreamingBody at 0x7f8cc8877b20>}

## Exercise: Asynchronous invocation 

Asynchronous invocations occur when a service invokes lambda but does not wait for a response. The two most popular services that utilize asynchronous invocations are S3 (the storage we've been using) and SNS (Simple Notification Service.) We'll be setting up asynchronous invocations on an S3 bucket for our preprocessing function.

Your task is to setup a trigger for the Lambda function we've been working  whenever a file is uploaded to a specific folder in S3. You will need to do the following:

* Create a new s3 folder within an existing bucket. 
* Create a new lambda trigger for S3 by clicking '+Add trigger'. Specifying the bucket. Specify a prefix of the desired folder. Specify a suffix of ".zip" to ensure that recursive calls don't occur. 
* Modify the lambda handler in the previous exercise using the starter code so that it properly parses the event that's sent to it. 

To test, upload reviews_Patio_Lawn_and_Garden_5.json.zip in this directory to your S3 bucket. 
To see if the lambda function is triggered, you can go to the Monitor tab. 



## Lambda Handler Starter Code: Parsing S3 Upload Event. 

In [ ]:
import urllib
from HelloBlazePreprocessLambda import preprocess

def lambda_handler(event, context):
    # Parse S3 event
    for r in event['Records']:
        bucket = r['s3']['bucket']['name']
        key = urllib.parse.unquote_plus(r['s3']['object']['key'], encoding='utf-8')
        uri = "/".join([bucket, key])
        # Preprocess newly uploaded data
        preprocess(uri)
    return {
        'statusCode': 200,
        'body': json.dumps("PREPROCESS SUCCESSFULLY!")
    }